In [1]:
import pandas as pd
import joblib
import random
import json
import sklearn

In [24]:
# !mkdir movies-mit-data/dicts/
# !mkdir movies-mit-data/nested-lists/
# !mkdir movies-mit-data/json/

In [2]:
# defining function to read files and store as nested lists

def read_bio(filepath):
    f = open(filepath, 'r')
    data = pd.DataFrame(f, columns=["line"])
    f.close()

    breaks = [0]
    breaks.extend(list(data[data.line=="\n"].index))
    labels = {}
    tokens = {}
    for i in range(len(breaks)-1):
        # print(i)
        labels[i] = []
        tokens[i] = []
        for j in range(breaks[i]+1, breaks[i+1]):
            line = data.loc[j, "line"]
            tokens[i].append(line.split("\t")[1][:-1])
            labels[i].append(line.split("\t")[0])

    return (list(tokens.values()), list(labels.values()))

In [5]:
# X, y = read_bio("/engtrain.bio")
X, y = read_bio("movies-mit-data/original-data/engtrain.bio")
X_test, y_test = read_bio("movies-mit-data/original-data/engtest.bio")
# trivia_X, trivia_y = read_bio("../datasets/movies/MIT_movie_NER/original_files/trivia10k13train.bio")
# trivia_X_test, trivia_y_test = read_bio("../datasets/movies/MIT_movie_NER/original_files/trivia10k13test.bio")

In [6]:
X

[['movies', 'star', 'bruce', 'willis'],
 ['show', 'me', 'films', 'with', 'drew', 'barrymore', 'from', 'the', '1980s'],
 ['what',
  'movies',
  'starred',
  'both',
  'al',
  'pacino',
  'and',
  'robert',
  'deniro'],
 ['find',
  'me',
  'all',
  'of',
  'the',
  'movies',
  'that',
  'starred',
  'harold',
  'ramis',
  'and',
  'bill',
  'murray'],
 ['find',
  'me',
  'a',
  'movie',
  'with',
  'a',
  'quote',
  'about',
  'baseball',
  'in',
  'it'],
 ['what', 'movies', 'have', 'mississippi', 'in', 'the', 'title'],
 ['show',
  'me',
  'science',
  'fiction',
  'films',
  'directed',
  'by',
  'steven',
  'spielberg'],
 ['do',
  'you',
  'have',
  'any',
  'thrillers',
  'directed',
  'by',
  'sofia',
  'coppola'],
 ['what',
  'leonard',
  'cohen',
  'songs',
  'have',
  'been',
  'used',
  'in',
  'a',
  'movie'],
 ['show', 'me', 'films', 'elvis', 'films', 'set', 'in', 'hawaii'],
 ['what', 'movie', 'is', 'references', 'zydrate'],
 ['are', 'there', 'any', 'musical', 'films', 'with', 

In [8]:
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=0)
X_test, y_test = shuffle(X_test, y_test, random_state=0)
# trivia_X, trivia_y = shuffle(trivia_X, trivia_y, random_state=0)
# trivia_X_test, trivia_y_test = shuffle(trivia_X_test, trivia_y_test, random_state=0)

val_cutoff = round(0.2*len(X))
X_val = X[:val_cutoff]
X_train = X[val_cutoff:]
y_val = y[:val_cutoff]
y_train = y[val_cutoff:]

In [12]:
joblib.dump(X_train, "movies-mit-data/nested-lists/X_train")
joblib.dump(y_train, "movies-mit-data/nested-lists/y_train")
joblib.dump(X_test, "movies-mit-data/nested-lists/X_test")
joblib.dump(y_test, "movies-mit-data/nested-lists/y_test")
joblib.dump(X_val, "movies-mit-data/nested-lists/X_val")
joblib.dump(y_val, "movies-mit-data/nested-lists/y_val")

['movies-mit-data/nested-lists/y_val']

In [14]:
training = {}
validation = {}
testing = {}
# trivia_training = {}
# trivia_validation = {}
# trivia_testing = {}

# trivia_training['sentences'] = trivia_X_train
# trivia_training['tags'] = trivia_y_train
# trivia_validation['sentences'] = trivia_X_val
# trivia_validation['tags'] = trivia_y_val
# trivia_testing['sentences'] = trivia_X_test
# trivia_testing['tags'] = trivia_y_test

training['sentences'] = X_train
training['tags'] = y_train
validation['sentences'] = X_val
validation['tags'] = y_val
testing['sentences'] = X_test
testing['tags'] = y_test


# joblib.dump(trivia_training, "../datasets/movies/MIT_movie_NER/dict_structure/trivia_training.dict")
# joblib.dump(trivia_validation, "../datasets/movies/MIT_movie_NER/dict_structure/trivia_validation.dict")
# joblib.dump(trivia_testing, "../datasets/movies/MIT_movie_NER/dict_structure/trivia_testing.dict")
joblib.dump(training, "movies-mit-data/dict/training.dict")
joblib.dump(validation, "movies-mit-data/dict/validation.dict")
joblib.dump(testing, "movies-mit-data/dict/testing.dict")

['movies-mit-data/dict/testing.dict']

In [16]:
X_train = pd.Series(X_train)
y_train = pd.Series(y_train)
X_test = pd.Series(X_test)
y_test = pd.Series(y_test)
X_val = pd.Series(X_val)
y_val = pd.Series(y_val)

In [17]:
mapper_list = ['B-Actor',
 'B-Award',
 'B-Character_Name',
 'B-Director',
 'B-Genre',
 'B-Opinion',
 'B-Origin',
 'B-Plot',
 'B-Quote',
 'B-Relationship',
 'B-Soundtrack',
 'B-Year',
 'I-Actor',
 'I-Award',
 'I-Character_Name',
 'I-Director',
 'I-Genre',
 'I-Opinion',
 'I-Origin',
 'I-Plot',
 'I-Quote',
 'I-Relationship',
 'I-Soundtrack',
 'I-Year',
 'O']

In [18]:
mapper = {}
i = 0
for each in mapper_list:
    mapper[each] = i
    i = i+1

In [19]:
plain_train = pd.DataFrame(columns=['tokens', 'ner_tags'])
plain_val = pd.DataFrame(columns=['tokens', 'ner_tags'])
plain_test = pd.DataFrame(columns=['tokens', 'ner_tags'])

In [20]:
plain_train["ner_tags"] = y_train
plain_test["ner_tags"] = y_test
plain_val["ner_tags"] = y_val

In [21]:
plain_train["tokens"] = X_train
plain_test["tokens"] = X_test
plain_val["tokens"] = X_val

In [23]:
plain_train.to_json("movies-mit-data/json/plain_training.json", orient='records', lines=True)
plain_test.to_json("movies-mit-data/json/plain_test.json", orient='records', lines=True)
plain_val.to_json(r"movies-mit-data/json/plain_val.json", orient='records', lines=True)